In [1]:
from experiment_project.utils.initial.util import init_sys_env
from experiment_project.utils.files.read import read_yaml
import dspy
init_sys_env()
secret_env_file = '/mnt/d/project/dy/extra/autogen/env_secret_config.yaml'

api_configs = read_yaml(secret_env_file)
model_config = api_configs.get('openai')
turbo = dspy.OpenAI(model=model_config.get('model'), max_tokens=1024,api_key=model_config.get('api_key'))
dspy.settings.configure(lm=turbo)

In [2]:
import dspy

# 定义 Signature
class BasicQA(dspy.Signature):
    """回答问题，答案要简短。"""
    question = dspy.InputField()
    answer = dspy.OutputField(desc="通常是 1 到 25 个词", prefix="问题的答案:")

# 创建任务模块
class QAModule(dspy.Module):
    def __init__(self):
        super().__init__()
        # 使用 BasicQA Signature 定义任务
        self.qa_task = dspy.Predict(BasicQA)

    def forward(self, question):
        # 执行任务并返回结果
        return self.qa_task(question=question)


# 创建 QAModule 实例
qa_module = QAModule()
# 定义问题
question = "成都最好吃的食物是什么？"
# 执行任务并获取答案
response = qa_module.forward(question)
print(response.answer)  # 输出答案，例如 "问题的答案: 珠穆朗玛峰"
turbo.inspect_history(n=1)
# 查看历史记录


In [5]:
turbo.history[0]['prompt']



In [30]:

# 创建任务模块
class QAModule(dspy.Module):
    def __init__(self):
        super().__init__()
        self.cot = dspy.ChainOfThought('question -> answer',n=5)
        # 使用 Predict 获取最终答案
    def forward(self, question):

        return self.cot(question= question)

# 创建 MathSolver 实例
qa_module = QAModule()

# 定义问题
problem = """
德国二战为什么战败? 使用中文回答
"""
# 解决问题
response = qa_module.forward(question=problem)
response
# 打印结果


In [31]:
turbo.history[-1]

## Retrieve 学习

In [27]:


import dspy

# 1. 选择 "搜索引擎":  这里使用 ColBERTv2 模型

colbert = dspy.ColBERTv2(url='http://20.102.90.50:2017/wiki17_abstracts') 
dspy.settings.configure(rm=colbert)  # 配置 DSPy 使用这个模型

# 2. 创建 Retrieve 对象,  指定返回 3 个结果
retriever = dspy.Retrieve(k=4) 

# 3.  输入问题，获取答案
question = "openai是什么公司？"
results = retriever(question).passages  # 获取相关段落

# 打印结果
for i, passage in enumerate(results):
    print(f"{i+1}. {passage}") 


In [37]:
import dspy



# 定义 Signature: 输入是两个字符串表示的数字，输出是它们的和
class AddTwoNumbers(dspy.Signature):
    a = dspy.InputField(desc="第一个数字", type=str)
    b = dspy.InputField(desc="第二个数字", type=str)
    sum = dspy.OutputField(desc="两个数字的和", type=str)

# 创建 PoT 对象
pot = dspy.ProgramOfThought(AddTwoNumbers,)

# 定义问题，将数字转换为字符串
problem = {"a": str(10), "b": str(20)}  

# 解决问题
response = pot(**problem)

# 打印结果
print(response.sum)  # 输出: 30


In [38]:
turbo.history[-1]

In [40]:

# 定义 Signature
class DayOfWeek(dspy.Signature):
    today = dspy.InputField(desc="今天是星期几", type=str)
    tomorrow = dspy.OutputField(desc="明天是星期几", type=str)

# 创建 CoT 对象
cot = dspy.ChainOfThought(DayOfWeek)

# 定义问题
problem = {"today": "星期一"}

# 解决问题
response = cot(**problem)

# # 打印 Chain of Thought 推理过程和结果
# print("Chain of Thought 推理过程:")
# for step in response.thoughts:
#     print(step)
# 
# print(f"最终答案: 明天是 {response.output.tomorrow}")


In [43]:
response

In [17]:
import dspy

# 定义一个简单的产品评论情感分析任务的签名
class ProductReviewSentiment(dspy.Signature):
    """分析产品评论的情感."""
    review = dspy.InputField(desc="")
    sentiment = dspy.OutputField(desc="说明评论的感情")

# 初始化 ReAct 模块，使用 ProductReviewSentiment 签名
react_module = dspy.ReAct(ProductReviewSentiment)

# 定义一个示例产品评论
review = "这款手机的相机太棒了！画质清晰，功能强大，我非常喜欢！但是我觉得它的续航不是特别好,用几个小时就需要充电!"

# 使用 ReAct 模块分析评论情感
result = react_module(review=review)

# 打印结果
print(f"评论: {review}")
print(f"情感分析结果: {result.sentiment}")



In [18]:
turbo.history[-1]

In [19]:

turbo.inspect_history(n=1)